# Import dataset

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets list
! pip install --upgrade --force-reinstall --no-deps kaggle

!kaggle competitions download -c nlp-getting-started -p 'dataset'

In [ ]:
cd dataset

In [ ]:
! unzip nlp-getting-started.zip

In [46]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [47]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [48]:
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [49]:
sample_submission

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


# EDA

In [50]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [51]:
train.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [52]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [53]:
test.isna().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

As we can see not every cell is filled. But what we are interested in is the text and target rows which are fully filled

# CountVectorizer

we vectorize text column of train dataset

In [54]:
from sklearn.feature_extraction.text import CountVectorizer

cntv = CountVectorizer()
vectxt_train = cntv.fit_transform(train['text'])

we do have 21637 unique strings in whole tweets bag

In [55]:
vectxt_train.toarray().shape

(7613, 21637)

In [56]:
vecdf_train = pd.DataFrame(data = vectxt_train.toarray(), columns = cntv.get_feature_names())
vecdf_train

,00,000,0000,007npen6lg,00cy9vxeff,00end,00pm,01,02,0215,02elqlopfk,02pm,03,030,033,034,039,03l7nwqdje,04,05,05th,06,060,061,06jst,07,073izwx0lb,08,0840728,0853,087809233445,0880,08lngclzsj,09,0abgfglh7x,0ajisa5531,0blkwcupzq,0btniwagt1,0bvk5tub4j,0c1y8g7e9p,...,ûªre,ûªs,ûªt,ûªve,ûï,ûïa,ûïafter,ûïairplane,ûïall,ûïcat,ûïdetonate,ûïfor,ûïhatchet,ûïi,ûïlittle,ûïlove,ûïmake,ûïnews,ûïnobody,ûïnumbers,ûïparties,ûïplans,ûïrichmond,ûïsippin,ûïstretcher,ûïthat,ûïthe,ûïwe,ûïwhen,ûïyou,ûò,ûò800000,ûòthe,ûòåêcnbc,ûó,ûóher,ûókody,ûónegligence,ûótech,ûówe
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7609,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7610,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7611,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Let's see how our strings perform

In [57]:
words_cnt = cntv.vocabulary_
sorted(words_cnt.items(), key=lambda x: x[1], reverse = True)  

[('ûówe', 21636),
 ('ûótech', 21635),
 ('ûónegligence', 21634),
 ('ûókody', 21633),
 ('ûóher', 21632),
 ('ûó', 21631),
 ('ûòåêcnbc', 21630),
 ('ûòthe', 21629),
 ('ûò800000', 21628),
 ('ûò', 21627),
 ('ûïyou', 21626),
 ('ûïwhen', 21625),
 ('ûïwe', 21624),
 ('ûïthe', 21623),
 ('ûïthat', 21622),
 ('ûïstretcher', 21621),
 ('ûïsippin', 21620),
 ('ûïrichmond', 21619),
 ('ûïplans', 21618),
 ('ûïparties', 21617),
 ('ûïnumbers', 21616),
 ('ûïnobody', 21615),
 ('ûïnews', 21614),
 ('ûïmake', 21613),
 ('ûïlove', 21612),
 ('ûïlittle', 21611),
 ('ûïi', 21610),
 ('ûïhatchet', 21609),
 ('ûïfor', 21608),
 ('ûïdetonate', 21607),
 ('ûïcat', 21606),
 ('ûïall', 21605),
 ('ûïairplane', 21604),
 ('ûïafter', 21603),
 ('ûïa', 21602),
 ('ûï', 21601),
 ('ûªve', 21600),
 ('ûªt', 21599),
 ('ûªs', 21598),
 ('ûªre', 21597),
 ('ûªm', 21596),
 ('ûªll', 21595),
 ('ûªd', 21594),
 ('ûªarabia', 21593),
 ('ûªa', 21592),
 ('ûª93', 21591),
 ('ûª', 21590),
 ('û_you', 21589),
 ('û_turns', 21588),
 ('û_that', 21587),
 ('û_t', 2

Let's transform our test 'text' column. But be aware we are using transform only since we want to use same model on test set and not to create second model what wouldn't have any sens.

In [58]:
vectxt_test = cntv.transform(test['text'])

In [59]:
vectxt_test.toarray().shape

(3263, 21637)

In [60]:
vecdf_test = pd.DataFrame(data = vectxt_test.toarray(), columns = cntv.get_feature_names())
vecdf_test

,00,000,0000,007npen6lg,00cy9vxeff,00end,00pm,01,02,0215,02elqlopfk,02pm,03,030,033,034,039,03l7nwqdje,04,05,05th,06,060,061,06jst,07,073izwx0lb,08,0840728,0853,087809233445,0880,08lngclzsj,09,0abgfglh7x,0ajisa5531,0blkwcupzq,0btniwagt1,0bvk5tub4j,0c1y8g7e9p,...,ûªre,ûªs,ûªt,ûªve,ûï,ûïa,ûïafter,ûïairplane,ûïall,ûïcat,ûïdetonate,ûïfor,ûïhatchet,ûïi,ûïlittle,ûïlove,ûïmake,ûïnews,ûïnobody,ûïnumbers,ûïparties,ûïplans,ûïrichmond,ûïsippin,ûïstretcher,ûïthat,ûïthe,ûïwe,ûïwhen,ûïyou,ûò,ûò800000,ûòthe,ûòåêcnbc,ûó,ûóher,ûókody,ûónegligence,ûótech,ûówe
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3259,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3260,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3261,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Naive Bayes vs Ridge Regression

Naive Bayes instance

In [61]:
from sklearn.naive_bayes import GaussianNB

gnb_model = GaussianNB()

Let's check how our model perfoms first

In [62]:
from sklearn import model_selection

scores = model_selection.cross_val_score(gnb_model, vectxt_train.toarray(), train["target"], cv=10, scoring="f1")
scores

array([0.60024301, 0.55220418, 0.54524089, 0.51489869, 0.58553387,
       0.55667506, 0.61407767, 0.60493827, 0.61179088, 0.61714286])

Ridge regression instance

In [63]:
from sklearn import linear_model

rc_model = linear_model.RidgeClassifier()

ridge regression performance

In [64]:
from sklearn import model_selection

scores = model_selection.cross_val_score(rc_model, vectxt_train, train["target"], cv=10, scoring="f1")
scores

array([0.6148532 , 0.48      , 0.44267516, 0.44748858, 0.52058824,
       0.56426332, 0.49911504, 0.47670251, 0.62611276, 0.69101124])

I will use ridge regression since it managed to predict up to 69% accurate

# Train

In [65]:
rc_model.fit(vectxt_train, train['target'])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [66]:
predictions = rc_model.predict(vectxt_test)
predictions

array([0, 1, 1, ..., 1, 1, 0])

# Submission

In [67]:
sample_submission['target'] = predictions
sample_submission['target'].value_counts()

0    2049
1    1214
Name: target, dtype: int64

In [68]:
sample_submission

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [69]:
sample_submission.to_csv('ss.csv', index = False)

In [70]:
ss = pd.read_csv('ss.csv')
ss

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


# Curiosity check:)

In [85]:
test['disaster_check'] = predictions
check = test[['text','disaster_check']]

text              Heard about #earthquake is different cities, stay safe everyone.
disaster_check                                                                   1
Name: 1, dtype: object

let's create a list of random 5 indexes from test.

In [100]:
from random import randint

rand_int = []

for i in range(5):
  rand_int.append(randint(0,len(test)))

rand_int

[2959, 2267, 2858, 831, 376]

display tweets and anwsers 

In [101]:
pd.set_option('display.max_colwidth', None) # to display full content

for i in rand_int:
  print(check.iloc[i,:])
  print('\n')

text              @Pandamoanimum Duly noted.The involuntary wince I have when people send me words like 'holibobs' must make it look like I have trapped wind.
disaster_check                                                                                                                                               0
Name: 2959, dtype: object


text              Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/z16jNX3LQe
disaster_check                                                                                                                                           1
Name: 2267, dtype: object


text              Cross-border terrorism: Pakistan caught red-handed again @timesofindia http://t.co/ZfdcWV4ch0
disaster_check                                                                                                1
Name: 2858, dtype: object


text              Double E! Just crushed that ball! #Jays
disast

It seems that the algorithm was able to predict correctly all of above examples. Of course it is arguable whether e.g. second sentence is a disaster. I mean this event is something that is probably already known by either police or medics. I think better example of (unexpected) disaster is 3rd sentence about terrorism because it can be unknown yet by the officials. Also quite cool example is 4th sentence where, despite the fact user used 'crushed' word in a tweet, algorithm correctly labeled it as 0.